In [16]:
import pandas as pd
import re
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import string
import ast
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)


# Get rid of duplicate ingredients in Ingredients_final
df['Ingredients_final'] = df['Ingredients_final'].apply(lambda x: ' '.join(set(x.split())))
tfidf = TfidfVectorizer() # Set TfidfVectorizer()
tfidf.fit(df['Ingredients_final'].values.astype('U')) # Create model
tfidf_recipe = tfidf.transform(df['Ingredients_final']) # Create encodings
# Define function that parses user input ingredients
def ingredient_parser(ingredients):
    measures = ['teaspoon', 't', 'tsp.', 'tsp', 'tablespoon', 'T', 'tbl.', 'tb', 'tbsp.','tbsp', 'fluid ounce', 'fl oz', 'gill', 'cup', 'c', 'pint', 'p', 'pt', 'fl pt', 'quart', 'q', 'qt', 'fl qt', 'gallon', 'g', 'gal', 'ml', 'milliliter', 'millilitre', 'cc', 'mL', 'l', 'liter', 'litre', 'L', 'dl', 'deciliter', 'decilitre', 'dL', 'bulb', 'level', 'heaped', 'rounded', 'whole', 'pinch', 'medium', 'slice', 'pound', 'lb', '#', 'ounce', 'oz', 'mg', 'milligram', 'milligramme', 'g', 'gram', 'gramme', 'kg', 'kilogram', 'kilogramme', 'x', 'of', 'mm', 'millimetre', 'millimeter', 'cm', 'centimeter', 'centimetre', 'm', 'meter', 'metre', 'inch', 'in', 'milli', 'centi', 'deci', 'hecto', 'kilo']
    bad_words = ['oil', 'fresh', 'olive', 'a', 'red', 'sauce', 'clove', 'or', 'pepper', 'bunch', 'salt', 'and', 'leaf', 'chilli', 'large', 'extra', 'water', 'white', 'ground', 'dried', 'sprig', 'small', 'free', 'handful', 'sugar', 'from', 'virgin', 'soy', 'black', 'chopped', 'vinegar', 'green', 'piece', 'seed', 'for', 'sustainable', 'range', 'cornstarch', 'higher', 'welfare', 'to', 'peeled', 'sesame', 'flour', 'tin', 'finely', 'the', 'freshly', 'bean', 'quality', 'few', 'ripe', 'parsley', 'sea', 'stock', 'source', 'flat', 'vegetable', 'smoked', 'organic', 'spring', 'fillet', 'sliced', 'plus', 'corn', 'plain', 'stick', 'cooking', 'light', 'picked', 'mixed', 'your', 'powder', 'bay', 'optional', 'baby', 'serve', 'stalk', 'unsalted', 'plum', 'natural', 'fat', 'fish', 'sweet', 'skin', 'such', 'juice', 'ask', 'brown', 'into', 'on', 'yellow', 'roughly', 'cut', 'good', 'dark', 'chili', 'orange', 'cherry', 'grated', 'frozen', 'bread', 'trimmed', 'breast', 'low', 'knob', 'dusting', 'salad', 'bell', 'cooked', 'runny', 'deseeded', 'balsamic', 'with', 'paste', 'bouillon', 'curry', 'streaky', 'use', 'pin', 'rasher', 'nut', 'cream', 'if', 'groundnut', 'soft', 'you', 'squash', 'tamari', 'chinese', 'zest', 'baking', 'grating', 'bone', 'hot', 'steak', 'boiling', 'minced', 'thigh', 'can', 'other', 'colour', 'shiitake', 'puree', 'dry', 'halved', 'skinless', 'spice', 'amount', 'chive', 'tinned', 'english', 'butternut', 'splash', 'shoulder', 'king', 'leftover', 'washed', 'firm', 'thick', 'flake', 'stir', 'broth', 'caper', 'big', 'dijon', 'is', 'little', 'pastry', 'five', 'sized', 'fishmonger', 'deep', 'removed', 'any', 'cube', 'frying', 'raw']

    translator = str.maketrans('', '', string.punctuation) # Get rid of punctuations using maketrans
    lemmatizer = WordNetLemmatizer() # Set lemmatizer
    new_ingred_list = [] # Empty list for parsed ingredients

    # Loop through each ingredient
    for i in ingredients:
        i.translate(translator) # Get rid of punctuation
        items = re.split(' |-', i) # Split with where there is a space or hyphenated words
        items = [word for word in items if word.isalpha()] # Get rid of non alphabet words
        items = [word.lower() for word in items] # Make everything lower case
        items = [unidecode.unidecode(word) for word in items] # Unidecode each word
        items = [lemmatizer.lemmatize(word) for word in items] # Lemmatize each word
        items = [word for word in items if word not in measures] # Take out measure words
        items = [word for word in items if word not in bad_words] # Take out bad words

        new_ingred_list.append(' '.join(items))# Append to list

    return ' '.join(new_ingred_list) # Return new parsed ingredients

# Define a function that gets cosine similarity scores with each recipe, taking in ingredient parameters
def get_scores(i):
    input_parsed = ingredient_parser(i)
    # Use pretrained tfidf model to encode our input ingredients
    ingredients_tfidf = tfidf.transform([input_parsed])
    # Calculate cosine similarity between actual recipe ingredients and test ingredients
    cos_sim = cosine_similarity(tfidf_recipe, ingredients_tfidf)
    return cos_sim
def get_recommendations(scores, N=5, calorie_limit=None, min_protein=None, min_fats=None, min_carbs=None):
    df_copy = df.copy()  # Copy the dataframe
    df_copy['Rec_score'] = scores  # Put scores into the dataframe

    # Filter by caloric limit if provided
    if calorie_limit is not None:
        df_copy = df_copy[df_copy['Calories'].astype(float) <= calorie_limit]

    # Filter by minimum protein if provided
    if min_protein is not None:
        df_copy = df_copy[df_copy['Protein/g'].astype(float) >= min_protein]

    # Filter by minimum fats if provided
    if min_fats is not None:
        df_copy = df_copy[df_copy['Fats/g'].astype(float) >= min_fats]

    # Filter by minimum carbs if provided
    if min_carbs is not None:
        df_copy = df_copy[df_copy['Carbs/g'].astype(float) >= min_carbs]

    # Order the scores with 'Rec_score' and sort to get the highest N scores
    top = df_copy.sort_values('Rec_score', ascending=False).head(N)

    # Create a dataframe to load in recommendations
    recommendation = top[['Title', 'Ingredients', 'Servings', 'Difficulty', 'Calories', 'Fats/g', 'Protein/g', 'Carbs/g', 'Time', 'URL', 'Rec_score']]
    return recommendation



# Assuming you have the necessary functions get_scores and get_recommendations already defined

# Function to get user input for caloric limit, minimum carbs, fats, and protein
def get_user_nutrient_limits():
    while True:
        try:
            calorie_limit = int(input('Enter the caloric limit for the recipes (or enter 0 for no limit): '))
            min_protein = int(input('Enter your minimum protein requirement (in grams): '))
            min_fats = int(input('Enter your minimum fats requirement (in grams): '))
            min_carbs = int(input('Enter your minimum carbs requirement (in grams): '))

            # Validate input
            if calorie_limit >= 0 and min_protein >= 0 and min_fats >= 0 and min_carbs >= 0:
                return calorie_limit, min_protein, min_fats, min_carbs
            else:
                print('Please enter non-negative values.')
        except ValueError:
            print('Please enter valid numbers.')

# Final function that takes in user ingredients and outputs N highest scoring recipes
def recipe_recommender():

    user_ingreds = []  # Empty list to store users ingredients
    user_input = ''  # Set user_input as nothing to enter while loop below

    while user_input != 'done':  # When user enters 'done', while loop exits
        user_input = input('Please enter an ingredient, when finished enter "done":').lower()  # Ask for input ingredient
        if user_input != 'done':  # Append to list if user_input is not 'done'
            user_ingreds.append(user_input)

    # Ask user for the number of recipes to be recommended
    while True:
        # If the number is given, break out of the while loop
        try:
            N = int(input('How many recipes would you like to be recommended?:'))
            break
        # If not a number, loop back and ask the user again
        except ValueError:
            print('Please enter a number.')

    # Get user input for caloric limit, minimum carbs, fats, and protein
    calorie_limit, min_protein, min_fats, min_carbs = get_user_nutrient_limits()

    # Calculate scores and recommendations
    scores = get_scores(user_ingreds)  # Get scores using get_scores() function
    recommendations = get_recommendations(scores, N, calorie_limit, min_protein, min_fats, min_carbs)  # Get recommendations using get_recommendations() function

    # Output results
    print(f'Ingredients inputted: {user_ingreds}')  # Print ingredients given
    print(f'Caloric Limit: {calorie_limit}')
    print(f'Minimum Protein: {min_protein}g')
    print(f'Minimum Fats: {min_fats}g')
    print(f'Minimum Carbs: {min_carbs}g')
    print('----------------------------------------')
    '''
    # Iterate through each recommendation
    for i, j in recommendations.iterrows():
        display(HTML(f"<b>{j[0]}</b>"))  # Display recipe name in bold

        # Loop through each ingredient
        for ingred in ast.literal_eval(j[1]):
            print(f'- {ingred}')  # Print ingredients in a bullet-pointed list
        print(f'Calories: {j["Calories"]}')  # Print calories
        print(f'Protein: {j["Protein/g"]}')
        print(f'Fats: {j["Fats/g"]}')
        print(f'Carbs: {j["Carbs/g"]}')

        print('----------------------------------------')'''

    del recommendations['Ingredients']  # Delete 'Ingredients' column as that is displayed above
    return recommendations  # Return the DataFrame with the rest of the information

# Example usage:
# recipe_recommender()

In [2]:
import pandas as pd
import re
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import string
import ast
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv('Cleaned_data.csv', index_col=False)

In [4]:
df.head()

,Unnamed: 0,Title,Ingredients,Servings,Difficulty,Calories,Fats/g,Protein/g,Carbs/g,Time,URL,Ingredients_parsed,Ingredients_final
0,0,Veggie pasta bake,"['½ x 170g loaf of garlic bread', 'olive oil', '2 cloves of garlic', '2 leeks', '1 bunch of fresh basil , (30g)', '2 teaspoons dried oregano', '3 x 400g tins of plum tomatoes', '500 g dried pasta , such as orecchiette, conchiglie or gemelli', '50 g Parmesan cheese , plus extra to serve', '8 cubes of frozen spinach']",8,Not too tricky,529.0,14.8,22.9,98.8,1 hour,https://www.jamieoliver.com/recipes/pasta-recipes/veggie-pasta-bake/,loaf garlic bread olive oil clove garlic leek bunch fresh basil dried oregano tin plum tomato dried pasta such a conchiglie or gemelli parmesan cheese plus extra to serve cube frozen spinach,loaf garlic garlic leek basil oregano tomato pasta conchiglie gemelli parmesan cheese spinach
1,2,Garlic chicken,"['4 cloves of garlic', '2 x 150g skinless free-range chicken breasts', '½ x 700g jar of chickpeas', '250 g baby spinach', '1 heaped teaspoon sumac']",2,Not too tricky,406.0,13.3,48.8,23.2,18 minutes,https://www.jamieoliver.com/recipes/chicken-recipes/garlic-chicken/,clove garlic skinless free range chicken breast jar chickpea baby spinach sumac,garlic chicken jar chickpea spinach sumac
2,3,Chicken & chips,"['1kg red-skinned potatoes', '2 onions', '1 bunch of oregano (20g)', '2 lemons', '4 x 250g free-range chicken legs']",4,Not too tricky,615.0,30.2,38.0,50.8,1 hour 18 minutes,https://www.jamieoliver.com/recipes/chicken-recipes/chicken-chips/,red skinned potato onion bunch oregano lemon free range chicken leg,skinned potato onion oregano lemon chicken
3,4,Sweet pea orecchiette,"['600g potatoes', '1 bunch of spring onions', '320g frozen peas', '300g dried orecchiette', '80g pecorino cheese, plus extra to serve']",4,Not too tricky,577.0,15.9,21.9,92.4,30 minutes,https://www.jamieoliver.com/recipes/pasta-recipes/sweet-pea-orecchiette/,potato bunch spring onion frozen pea dried orecchiette pecorino plus extra to serve,potato onion pea orecchiette pecorino
4,5,Island salad,"['1 cucumber', '2 pittas', '1 x 415g tin of sliced peaches in juice', '120g mixed salad leaves', '60g halloumi cheese']",2,Not too tricky,361.0,15.2,15.9,39.8,15 minutes,https://www.jamieoliver.com/recipes/fruit-recipes/island-salad/,cucumber pitta tin sliced peach juice mixed salad leaf halloumi cheese,cucumber pitta peach halloumi cheese


In [5]:
del df['Unnamed: 0'] # Delete 'Unnamed: 0' column

In [6]:
df.head()

,Title,Ingredients,Servings,Difficulty,Calories,Fats/g,Protein/g,Carbs/g,Time,URL,Ingredients_parsed,Ingredients_final
0,Veggie pasta bake,"['½ x 170g loaf of garlic bread', 'olive oil', '2 cloves of garlic', '2 leeks', '1 bunch of fresh basil , (30g)', '2 teaspoons dried oregano', '3 x 400g tins of plum tomatoes', '500 g dried pasta , such as orecchiette, conchiglie or gemelli', '50 g Parmesan cheese , plus extra to serve', '8 cubes of frozen spinach']",8,Not too tricky,529.0,14.8,22.9,98.8,1 hour,https://www.jamieoliver.com/recipes/pasta-recipes/veggie-pasta-bake/,loaf garlic bread olive oil clove garlic leek bunch fresh basil dried oregano tin plum tomato dried pasta such a conchiglie or gemelli parmesan cheese plus extra to serve cube frozen spinach,loaf garlic garlic leek basil oregano tomato pasta conchiglie gemelli parmesan cheese spinach
1,Garlic chicken,"['4 cloves of garlic', '2 x 150g skinless free-range chicken breasts', '½ x 700g jar of chickpeas', '250 g baby spinach', '1 heaped teaspoon sumac']",2,Not too tricky,406.0,13.3,48.8,23.2,18 minutes,https://www.jamieoliver.com/recipes/chicken-recipes/garlic-chicken/,clove garlic skinless free range chicken breast jar chickpea baby spinach sumac,garlic chicken jar chickpea spinach sumac
2,Chicken & chips,"['1kg red-skinned potatoes', '2 onions', '1 bunch of oregano (20g)', '2 lemons', '4 x 250g free-range chicken legs']",4,Not too tricky,615.0,30.2,38.0,50.8,1 hour 18 minutes,https://www.jamieoliver.com/recipes/chicken-recipes/chicken-chips/,red skinned potato onion bunch oregano lemon free range chicken leg,skinned potato onion oregano lemon chicken
3,Sweet pea orecchiette,"['600g potatoes', '1 bunch of spring onions', '320g frozen peas', '300g dried orecchiette', '80g pecorino cheese, plus extra to serve']",4,Not too tricky,577.0,15.9,21.9,92.4,30 minutes,https://www.jamieoliver.com/recipes/pasta-recipes/sweet-pea-orecchiette/,potato bunch spring onion frozen pea dried orecchiette pecorino plus extra to serve,potato onion pea orecchiette pecorino
4,Island salad,"['1 cucumber', '2 pittas', '1 x 415g tin of sliced peaches in juice', '120g mixed salad leaves', '60g halloumi cheese']",2,Not too tricky,361.0,15.2,15.9,39.8,15 minutes,https://www.jamieoliver.com/recipes/fruit-recipes/island-salad/,cucumber pitta tin sliced peach juice mixed salad leaf halloumi cheese,cucumber pitta peach halloumi cheese


In [8]:
pd.set_option('display.max_colwidth', None) #Allows for full visual of URL.


# Get rid of duplicate ingredients in Ingredients_final
df['Ingredients_final'] = df['Ingredients_final'].apply(lambda x: ' '.join(set(x.split())))
tfidf = TfidfVectorizer() # Set TfidfVectorizer()
tfidf.fit(df['Ingredients_final'].values.astype('U')) # Create model
tfidf_recipe = tfidf.transform(df['Ingredients_final']) # Create encodings
# Define function that parses user input ingredients
def ingredient_parser(ingredients):
    measures = ['teaspoon', 't', 'tsp.', 'tsp', 'tablespoon', 'T', 'tbl.', 'tb', 'tbsp.','tbsp', 'fluid ounce', 'fl oz', 'gill', 'cup', 'c', 'pint', 'p', 'pt', 'fl pt', 'quart', 'q', 'qt', 'fl qt', 'gallon', 'g', 'gal', 'ml', 'milliliter', 'millilitre', 'cc', 'mL', 'l', 'liter', 'litre', 'L', 'dl', 'deciliter', 'decilitre', 'dL', 'bulb', 'level', 'heaped', 'rounded', 'whole', 'pinch', 'medium', 'slice', 'pound', 'lb', '#', 'ounce', 'oz', 'mg', 'milligram', 'milligramme', 'g', 'gram', 'gramme', 'kg', 'kilogram', 'kilogramme', 'x', 'of', 'mm', 'millimetre', 'millimeter', 'cm', 'centimeter', 'centimetre', 'm', 'meter', 'metre', 'inch', 'in', 'milli', 'centi', 'deci', 'hecto', 'kilo']
    bad_words = ['oil', 'fresh', 'olive', 'a', 'red', 'sauce', 'clove', 'or', 'pepper', 'bunch', 'salt', 'and', 'leaf', 'chilli', 'large', 'extra', 'water', 'white', 'ground', 'dried', 'sprig', 'small', 'free', 'handful', 'sugar', 'from', 'virgin', 'soy', 'black', 'chopped', 'vinegar', 'green', 'piece', 'seed', 'for', 'sustainable', 'range', 'cornstarch', 'higher', 'welfare', 'to', 'peeled', 'sesame', 'flour', 'tin', 'finely', 'the', 'freshly', 'bean', 'quality', 'few', 'ripe', 'parsley', 'sea', 'stock', 'source', 'flat', 'vegetable', 'smoked', 'organic', 'spring', 'fillet', 'sliced', 'plus', 'corn', 'plain', 'stick', 'cooking', 'light', 'picked', 'mixed', 'your', 'powder', 'bay', 'optional', 'baby', 'serve', 'stalk', 'unsalted', 'plum', 'natural', 'fat', 'fish', 'sweet', 'skin', 'such', 'juice', 'ask', 'brown', 'into', 'on', 'yellow', 'roughly', 'cut', 'good', 'dark', 'chili', 'orange', 'cherry', 'grated', 'frozen', 'bread', 'trimmed', 'breast', 'low', 'knob', 'dusting', 'salad', 'bell', 'cooked', 'runny', 'deseeded', 'balsamic', 'with', 'paste', 'bouillon', 'curry', 'streaky', 'use', 'pin', 'rasher', 'nut', 'cream', 'if', 'groundnut', 'soft', 'you', 'squash', 'tamari', 'chinese', 'zest', 'baking', 'grating', 'bone', 'hot', 'steak', 'boiling', 'minced', 'thigh', 'can', 'other', 'colour', 'shiitake', 'puree', 'dry', 'halved', 'skinless', 'spice', 'amount', 'chive', 'tinned', 'english', 'butternut', 'splash', 'shoulder', 'king', 'leftover', 'washed', 'firm', 'thick', 'flake', 'stir', 'broth', 'caper', 'big', 'dijon', 'is', 'little', 'pastry', 'five', 'sized', 'fishmonger', 'deep', 'removed', 'any', 'cube', 'frying', 'raw']

    translator = str.maketrans('', '', string.punctuation) # Get rid of punctuations using maketrans
    lemmatizer = WordNetLemmatizer() # Set lemmatizer
    new_ingred_list = [] # Empty list for parsed ingredients

    # Loop through each ingredient
    for i in ingredients:
        i.translate(translator) # Get rid of punctuation
        items = re.split(' |-', i) # Split with where there is a space or hyphenated words
        items = [word for word in items if word.isalpha()] # Get rid of non alphabet words
        items = [word.lower() for word in items] # Make everything lower case
        items = [unidecode.unidecode(word) for word in items] # Unidecode each word
        items = [lemmatizer.lemmatize(word) for word in items] # Lemmatize each word
        items = [word for word in items if word not in measures] # Take out measure words
        items = [word for word in items if word not in bad_words] # Take out bad words

        new_ingred_list.append(' '.join(items))# Append to list

    return ' '.join(new_ingred_list) # Return new parsed ingredients

# Define a function that gets cosine similarity scores with each recipe, taking in ingredient parameters
def get_scores(i):
    input_parsed = ingredient_parser(i)
    # Use pretrained tfidf model to encode our input ingredients
    ingredients_tfidf = tfidf.transform([input_parsed])
    # Calculate cosine similarity between actual recipe ingredients and test ingredients
    cos_sim = cosine_similarity(tfidf_recipe, ingredients_tfidf)
    return cos_sim
def get_recommendations(scores, N=5, calorie_limit=None, min_protein=None, min_fats=None, min_carbs=None):
    df_copy = df.copy()  # Copy the dataframe
    df_copy['Rec_score'] = scores  # Put scores into the dataframe

    # Filter by caloric limit if provided
    if calorie_limit is not None:
        df_copy = df_copy[df_copy['Calories'].astype(float) <= calorie_limit]

    # Filter by minimum protein if provided
    if min_protein is not None:
        df_copy = df_copy[df_copy['Protein/g'].astype(float) >= min_protein]

    # Filter by minimum fats if provided
    if min_fats is not None:
        df_copy = df_copy[df_copy['Fats/g'].astype(float) >= min_fats]

    # Filter by minimum carbs if provided
    if min_carbs is not None:
        df_copy = df_copy[df_copy['Carbs/g'].astype(float) >= min_carbs]

    # Order the scores with 'Rec_score' and sort to get the highest N scores
    top = df_copy.sort_values('Rec_score', ascending=False).head(N)

    # Create a dataframe to load in recommendations
    recommendation = top[['Title', 'Ingredients', 'Servings', 'Difficulty', 'Calories', 'Fats/g', 'Protein/g', 'Carbs/g', 'Time', 'URL', 'Rec_score']]
    return recommendation



# Assuming you have the necessary functions get_scores and get_recommendations already defined

# Function to get user input for caloric limit, minimum carbs, fats, and protein
def get_user_nutrient_limits():
    while True:
        try:
            calorie_limit = int(input('Enter the caloric limit for the recipes (or enter 0 for no limit): '))
            min_protein = int(input('Enter your minimum protein requirement (in grams): '))
            min_fats = int(input('Enter your minimum fats requirement (in grams): '))
            min_carbs = int(input('Enter your minimum carbs requirement (in grams): '))

            # Validate input
            if calorie_limit >= 0 and min_protein >= 0 and min_fats >= 0 and min_carbs >= 0:
                return calorie_limit, min_protein, min_fats, min_carbs
            else:
                print('Please enter non-negative values.')
        except ValueError:
            print('Please enter valid numbers.')

# Final function that takes in user ingredients and outputs N highest scoring recipes
def recipe_recommender():

    user_ingreds = []  # Empty list to store users ingredients
    user_input = ''  # Set user_input as nothing to enter while loop below

    while user_input != 'done':  # When user enters 'done', while loop exits
        user_input = input('Please enter an ingredient, when finished enter "done":').lower()  # Ask for input ingredient
        if user_input != 'done':  # Append to list if user_input is not 'done'
            user_ingreds.append(user_input)

    # Ask user for the number of recipes to be recommended
    while True:
        # If the number is given, break out of the while loop
        try:
            N = int(input('How many recipes would you like to be recommended?:'))
            break
        # If not a number, loop back and ask the user again
        except ValueError:
            print('Please enter a number.')

    # Get user input for caloric limit, minimum carbs, fats, and protein
    calorie_limit, min_protein, min_fats, min_carbs = get_user_nutrient_limits()

    # Calculate scores and recommendations
    scores = get_scores(user_ingreds)  # Get scores using get_scores() function
    recommendations = get_recommendations(scores, N, calorie_limit, min_protein, min_fats, min_carbs)  # Get recommendations using get_recommendations() function

    # Output results
    print(f'Ingredients inputted: {user_ingreds}')  # Print ingredients given
    print(f'Caloric Limit: {calorie_limit}')
    print(f'Minimum Protein: {min_protein}g')
    print(f'Minimum Fats: {min_fats}g')
    print(f'Minimum Carbs: {min_carbs}g')
    print('----------------------------------------')
    '''
    # Iterate through each recommendation
    for i, j in recommendations.iterrows():
        display(HTML(f"<b>{j[0]}</b>"))  # Display recipe name in bold

        # Loop through each ingredient
        for ingred in ast.literal_eval(j[1]):
            print(f'- {ingred}')  # Print ingredients in a bullet-pointed list
        print(f'Calories: {j["Calories"]}')  # Print calories
        print(f'Protein: {j["Protein/g"]}')
        print(f'Fats: {j["Fats/g"]}')
        print(f'Carbs: {j["Carbs/g"]}')

        print('----------------------------------------')'''

    del recommendations['Ingredients']  # Delete 'Ingredients' column as that is displayed above
    return recommendations  # Return the DataFrame with the rest of the information

# Example usage:
# recipe_recommender()

In [9]:
recipe_recommender()

Ingredients inputted: ['egg', 'rice']
Caloric Limit: 1000
Minimum Protein: 10g
Minimum Fats: 10g
Minimum Carbs: 10g
----------------------------------------


,Title,Servings,Difficulty,Calories,Fats/g,Protein/g,Carbs/g,Time,URL,Rec_score
213,Tah chin,6,Showing off,741.0,33.4,36.4,73.8,3 hours 50 minutes,https://www.jamieoliver.com/recipes/lamb-recipes/tah-chin/,0.384577
257,"Pea, mint & basil arancini cakes",4,Not too tricky,744.0,48.5,16.0,56.4,50 minutes plus cooling,https://www.jamieoliver.com/recipes/rice-recipes/pea-mint-amp-basil-arancini-cakes/,0.317683
399,Tacu tacu,4,Super easy,476.0,10.8,18.4,74.8,1 hour,https://www.jamieoliver.com/recipes/rice-recipes/tacu-tacu/,0.317518
574,Rescue me kedgeree,4,Not too tricky,494.0,14.4,36.6,54.6,35 minutes,https://www.jamieoliver.com/recipes/fish-recipes/rescue-me-kedgeree/,0.300962
590,Mexican breakfast,6,Super easy,218.0,11.1,10.1,19.5,40 minutes,https://www.jamieoliver.com/recipes/eggs-recipes/mexican-breakfast/,0.286761
